# Applied Data Science Capstone Notebook - create coordinates list
In this notebook I have used BeautifulSoup to scrape the wikipedia website for the table content.
This content is traversed through and added to python lists which are added to the dataframe in the last 3 lines
Since the table no longer has duplicate postal codes I have not needed to implement the code to concatenate neighborhoods (it has already been done).
Once build, the Geocoder library is used to augment it with lattitude and longtitude information.

### First import the python libraries

In [2]:
!pip install geocoder
!pip install folium

import pandas as pd
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from urllib.request import Request
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

     |████████████████████████████████| 102kB 17.8MB/s ta 0:00:01


### Use BeautifulSoup to get the table content of the wikipedia page and

In [3]:
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
table = soup.find("table", class_="wikitable sortable")
rows = table.find_all('tr')

### Create the lists and the dataframe needed

In [4]:
df_table = pd.DataFrame()
geolocator = Nominatim()

pCodes = []
boroughs = []
neighborhoods = []
lattitudes = []
longtitudes = []

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


### Traverse through the data and build the lists containing the content

In [5]:
for row in rows:
    fields = row.find_all("td")
    content_list = []
    
    for field in fields:
        content = field.get_text().split('\n')
        content_list.append(content[0])
    if len(content_list) > 0:
        if content_list[1] != 'Not assigned':
            pCodes.append(content_list[0])
            boroughs.append(content_list[1])
            location = geolocator.geocode(content_list[1], ', ' + content_list[2] + ', Toronto, Canada')
            lattitudes.append(location.raw['lat'])
            longtitudes.append(location.raw['lon'])
            if content_list[2] != 'Not assigned':
                neighborhoods.append(content_list[2])
            else:
                neighborhoods.append(content_list[1])

### Finally move the lists into the dataframe

In [6]:
df_table["Postal Code"] = pCodes
df_table["Borough"] = boroughs
df_table["Neighborhood"] = neighborhoods
df_table["Lattitude"] = lattitudes
df_table["Longtitude"] = longtitudes

### Build the dataframe

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

df_table

,Postal Code,Borough,Neighborhood,Lattitude,Longtitude
0,M3A,North York,Parkwoods,43.7543263,-79.44911696639593
1,M4A,North York,Victoria Village,43.7543263,-79.44911696639593
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6541737,-79.38081164513409
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543263,-79.44911696639593
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6541737,-79.38081164513409
5,M9A,Etobicoke,Islington Avenue,43.671459150000004,-79.55249206611668
6,M1B,Scarborough,"Malvern, Rouge",54.2820009,-0.4011868
7,M3B,North York,Don Mills,43.7543263,-79.44911696639593
8,M4B,East York,"Parkview Hill, Woodbine Gardens",39.9737093,-76.6863554
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6541737,-79.38081164513409


### Finally output the shape of the table

In [13]:
print(df_table.shape)

(103, 5)


### Display a map of the city with markers on the areas

In [21]:
address = 'Toronto, Canada'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_table['Lattitude'], df_table['Longtitude'], df_table['Borough'], df_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Define Foursquare Credentials and Version

In [23]:
#CLIENT_ID = ''
#CLIENT_SECRET = ''
VERSION = '20200510'

In [54]:
df_toronto_subset = pd.DataFrame()

pCodes = []
boroughs = []
neighborhoods = []
lattitudes = []
longtitudes = []

for idx, row in df_table.iterrows():
    if row['Borough'].find('Toronto') > -1:
        pCodes.append(row[0])
        boroughs.append(row[1])
        neighborhoods.append(row[2])
        lattitudes.append(row[3])
        longtitudes.append(row[4])

df_toronto_subset["Postal Code"] = pCodes
df_toronto_subset["Borough"] = boroughs
df_toronto_subset["Neighborhood"] = neighborhoods
df_toronto_subset["Lattitude"] = lattitudes
df_toronto_subset["Longtitude"] = longtitudes

df_toronto_subset

,Postal Code,Borough,Neighborhood,Lattitude,Longtitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6541737,-79.38081164513409
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6541737,-79.38081164513409
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6541737,-79.38081164513409
3,M5C,Downtown Toronto,St. James Town,43.6541737,-79.38081164513409
4,M4E,East Toronto,The Beaches,43.6534817,-79.3839347
5,M5E,Downtown Toronto,Berczy Park,43.6541737,-79.38081164513409
6,M5G,Downtown Toronto,Central Bay Street,43.6541737,-79.38081164513409
7,M6G,Downtown Toronto,Christie,43.6541737,-79.38081164513409
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6541737,-79.38081164513409
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6534817,-79.3839347


In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:

toronto_venues = getNearbyVenues(names=df_toronto_subset['Neighborhood'],
                                   latitudes=df_toronto_subset['Lattitude'],
                                   longitudes=df_toronto_subset['Longtitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

### display info about venues

In [66]:
print(toronto_venues.shape)

toronto_venues.head()

(3264, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
1,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
2,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,Ed Mirvish Theatre,43.655102,-79.379768,Theater
3,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,Indigo,43.653515,-79.380696,Bookstore
4,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,LUSH,43.653557,-79.380400,Cosmetics Shop


In [65]:
toronto_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 68 uniques categories.


### Analyze the neighborhoods

In [67]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Women's Store,American Restaurant,Art Gallery,Art Museum,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Café,Camera Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Diner,Ethiopian Restaurant,Fast Food Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Travel,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Opera House,Other Great Outdoors,Performing Arts Venue,Plaza,Poke Place,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


,Neighborhood,Women's Store,American Restaurant,Art Gallery,Art Museum,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Café,Camera Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Diner,Ethiopian Restaurant,Fast Food Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Travel,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Opera House,Other Great Outdoors,Performing Arts Venue,Plaza,Poke Place,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Berczy Park,0.010417,0.010417,0.010417,0.010417,0.010417,0.020833,0.010417,0.010417,0.010417,0.010417,0.031250,0.010417,0.010417,0.062500,0.010417,0.104167,0.010417,0.000000,0.010417,0.000000,0.020833,0.031250,0.010417,0.010417,0.010417,0.020833,0.010417,0.000000,0.010417,0.010417,0.000000,0.041667,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.031250,0.010417,0.010417,0.010417,0.010417,0.010417,0.000000,0.010417,0.010417,0.020833,0.010417,0.010417,0.020833,0.041667,0.000000,0.03125,0.020833,0.010417,0.010417,0.000000,0.010417,0.020833,0.010417,0.010417,0.020833,0.000000,0.010417,0.010417
1,"Brockton, Parkdale Village, Exhibition Place",0.013889,0.027778,0.000000,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.000000,0.027778,0.000000,0.000000,0.083333,0.013889,0.069444,0.000000,0.013889,0.013889,0.013889,0.027778,0.027778,0.000000,0.013889,0.013889,0.013889,0.000000,0.013889,0.000000,0.000000,0.013889,0.027778,0.013889,0.000000,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.000000,0.000000,0.027778,0.013889,0.000000,0.013889,0.041667,0.013889,0.00000,0.027778,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.027778,0.027778,0.013889,0.013889,0.013889
2,Business reply mail Processing Centre,0.013889,0.027778,0.000000,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.000000,0.027778,0.000000,0.000000,0.083333,0.013889,0.069444,0.000000,0.013889,0.013889,0.013889,0.027778,0.027778,0.000000,0.013889,0.013889,0.013889,0.000000,0.013889,0.000000,0.000000,0.013889,0.027778,0.013889,0.000000,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.000000,0.000000,0.027778,0.013889,0.000000,0.013889,0.041667,0.013889,0.00000,0.027778,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.027778,0.027778,0.013889,0.013889,0.013889
3,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0.010417,0.010417,0.010417,0.010417,0.010417,0.020833,0.010417,0.010417,0.010417,0.010417,0.031250,0.010417,0.010417,0.062500,0.010417,0.104167,0.010417,0.000000,0.010417,0.000000,0.020833,0.031250,0.010417,0.010417,0.010417,0.020833,0.010417,0.000000,0.010417,0.010417,0.000000,0.041667,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.031250,0.010417,0.010417,0.010417,0.010417,0.010417,0.000000,0.010417,0.010417,0.020833,0.010417,0.010417,0.020833,0.041667,0.000000,0.03125,0.020833,0.010417,0.010417,0.000000,0.010417,0.020833,0.010417,0.010417,0.020833,0.000000,0.010417,0.010417
4,Central Bay Street,0.010417,0.010417,0.010417,0.010417,0.010417,0.020833,0.010417,0.010417,0.010417,0.010417,0.031250,0.010417,0.010417,0.062500,0.010417,0.104167,0.010417,0.000000,0.010417,0.000000,0.020833,0.031250,0.010417,0.010417,0.010417,0.020833,0.010417,0.000000,0.010417,0.010417,0.000000,0.041667,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.03125

### 5 most common venues

In [72]:
num_top_venues = 5

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Coffee Shop     0.10
1  Clothing Store  0.06
2  Hotel           0.04
3  Restaurant      0.04
4  Diner           0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0  Clothing Store  0.08
1  Coffee Shop     0.07
2  Restaurant      0.04
3  Café            0.03
4  Hotel           0.03


----Business reply mail Processing Centre----
            venue  freq
0  Clothing Store  0.08
1  Coffee Shop     0.07
2  Restaurant      0.04
3  Café            0.03
4  Hotel           0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
            venue  freq
0  Coffee Shop     0.10
1  Clothing Store  0.06
2  Hotel           0.04
3  Restaurant      0.04
4  Diner           0.03


----Central Bay Street----
            venue  freq
0  Coffee Shop     0.10
1  Clothing Store  0.06
2  Hotel           0.04
3  Restaurant      0.04
4  Diner           0.03


----Chris

### top 10 venues for each neighborhood.

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
1,"Brockton, Parkdale Village, Exhibition Place",Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Diner,Seafood Restaurant,Café,Plaza,Hotel,Thai Restaurant
2,Business reply mail Processing Centre,Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Diner,Seafood Restaurant,Café,Plaza,Hotel,Thai Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop


### K-means clustering of neighborhoods

In [78]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1], dtype=int32)

### Show Toronto data with top venues

In [82]:
toronto_merged = df_toronto_subset
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Lattitude,Longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6541737,-79.38081164513409,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6541737,-79.38081164513409,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6541737,-79.38081164513409,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.6541737,-79.38081164513409,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Diner,Middle Eastern Restaurant,Café,Gastropub,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.6534817,-79.3839347,1,Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Diner,Seafood Restaurant,Café,Plaza,Hotel,Thai Restaurant


### Show on Folium map

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Lattitude'], toronto_merged['Longtitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters